In [46]:
import pandas as pd
dataset_name = 'en'
df = pd.read_csv(f'data/{dataset_name}/job_task.csv', sep='\t')
occupation_candidate = pd.read_parquet(f'../step4-title_occupation/data/{dataset_name}/seed0/input0/occupation_candidate.parquet')[['title', 'occupation_candidate']]
occupation_candidate.columns = ['job title', 'occupation_candidate']
occupation_candidate
df = pd.merge(df, occupation_candidate)
df

,job title,job description,occupation,task,similarity,occupation_candidate
0,Customer Support Specialists,Support at NexHealth\n\nIt is an exciting time...,Customer Service Representatives,Contact customers to respond to inquiries or t...,0.927690,"[Customer Service Representatives, Project Man..."
1,Quality Control Analyst,Coordinate calibration schedules of assets and...,Quality Control Analysts,"Review balance sheets, operating income and ex...",0.912580,"[Quality Control Analysts, Project Management ..."
2,Quality Control Analyst,Routine Environmental Monitoring and utilities...,Quality Control Analysts,"Write technical reports or documentation, such...",0.912580,"[Quality Control Analysts, Project Management ..."
3,Quality Control Analyst,Coordinate calibration schedules of assets and...,Quality Control Analysts,Ensure that lab cleanliness and safety standar...,0.912580,"[Quality Control Analysts, Project Management ..."
4,Construction Inspectors,An Inspector may be required to oversee multip...,Construction and Building Inspectors,Create Web models or prototypes that include p...,0.909876,"[Transportation Inspectors, Aviation Inspector..."
...,...,...,...,...,...,...
97750,Azure Infra Lead,Implement core networking infrastructureImplem...,Armored Assault Vehicle Officers,Create or review technical design documentatio...,0.481020,"[Armored Assault Vehicle Officers, Military Of..."
97751,cross-border e-commerce manager（跨境购经理）,"Monitor and analyze sales data, timely adjust ...","Electronic Equipment Installers and Repairers,...",Troubleshoot network or connectivity problems ...,0.418025,"[Helpers--Pipelayers, Plumbers, Pipefitters, a..."
97752,지역 영업 관리자,The ideal candidate will prospect and close sa...,Biofuels/Biodiesel Technology and Product Deve...,"Propose new biofuels products, processes, tech...",0.369749,[Criminal Justice and Law Enforcement Teachers...
97753,海外社媒运营,"Manage content operations, planning, and publi...",Financial Quantitative Analysts,Conduct site audits to ensure adherence to saf...,0.356020,"[Musicians and Singers, Medical Secretaries an..."


In [47]:
df_group = df.groupby('job title').agg(list)
df_group['task'] = df_group['task'].apply(lambda x: list(set(x)))
df_group['task num'] = df_group['task'].apply(len)
df_group['job description'] = df_group['job description'].apply(lambda x: list(set(x)))
df_group['job num'] = df_group['job description'].apply(len)
df_group = df_group[['job description','job num', 'task', 'task num']].reset_index()
final_df = pd.merge(df.drop(['job description', 'task'], axis=1).drop_duplicates(['occupation', 'job title']), df_group)
final_df = final_df.sort_values(['job num', 'similarity'], ascending=[False, False])

suffix_list = final_df['job title'].apply(lambda x: x.split(' ')[-1]).tolist()
suffix_list = list(set([i for i in suffix_list if i[-2:] in ['er', 'or'] or i[-4:] in ['cian', 'list', 'tant', 'lyst']]))
len(suffix_list)
final_df['title_prefix'] = final_df['job title'].apply(lambda x: ' '.join(x.split(' ')[:-1]) if x.split(' ')[-1] in suffix_list else x)
final_df['title_suffix'] = final_df['job title'].apply(lambda x: x.split(' ')[-1] if x.split(' ')[-1] in suffix_list else '')
final_df = final_df.set_index('title_suffix').reset_index()
final_df = final_df.set_index('title_prefix').reset_index()
final_df = final_df.sort_values(['title_suffix', 'similarity'])
final_df.to_excel(f'data/{dataset_name}/{dataset_name}_job_tile_annotated.xlsx', index=None)
final_df[final_df['similarity']>0.8].to_excel(f'data/{dataset_name}/{dataset_name}_job_tile_annotated_sim0.8.xlsx', index=None)
final_df[(final_df['job num']>1) & (final_df['similarity']>0.8)].to_excel(f'data/{dataset_name}/{dataset_name}_job_tile_annotated_sim0.8_num1.xlsx', index=None)
final_df

,title_prefix,title_suffix,job title,occupation,similarity,occupation_candidate,job description,job num,task,task num
32506,American Vice President(美洲区域副总裁),,American Vice President(美洲区域副总裁),"Judges, Magistrate Judges, and Magistrates",0.346746,"[Airline Pilots, Copilots, and Flight Engineer...",[Complete other tasks assigned by group and re...,1,"[Identify, review, or select vendors or consul...",1
32505,海外社媒运营,,海外社媒运营,Financial Quantitative Analysts,0.356020,"[Musicians and Singers, Medical Secretaries an...","[Manage content operations, planning, and publ...",1,[Conduct site audits to ensure adherence to sa...,1
32504,지역 영업 관리자,,지역 영업 관리자,Biofuels/Biodiesel Technology and Product Deve...,0.369749,[Criminal Justice and Law Enforcement Teachers...,[The ideal candidate will prospect and close s...,1,"[Propose new biofuels products, processes, tec...",1
32503,cross-border e-commerce manager（跨境购经理）,,cross-border e-commerce manager（跨境购经理）,"Electronic Equipment Installers and Repairers,...",0.418025,"[Helpers--Pipelayers, Plumbers, Pipefitters, a...","[Monitor and analyze sales data, timely adjust...",1,[Troubleshoot network or connectivity problems...,1
32502,Azure Infra Lead,,Azure Infra Lead,Armored Assault Vehicle Officers,0.481020,"[Armored Assault Vehicle Officers, Military Of...",[Implement core networking infrastructureImple...,1,[Create or review technical design documentati...,1
...,...,...,...,...,...,...,...,...,...,...
5295,tower,technician,tower technician,Wind Turbine Service Technicians,0.850964,"[Lighting Technicians, Pile Driver Operators, ...",[Complete work in a timely and efficient manne...,4,"[Climb wind turbine towers to inspect, maintai...",4
20064,Cable Technician Data center,technician,Cable Technician Data center technician,Computer Network Support Specialists,0.861907,"[Wind Turbine Service Technicians, Signal and ...",[You serve as the initial escalation point for...,1,"[Identify the causes of networking problems, u...",1
19016,Corrosion,technician,Corrosion technician,Wind Turbine Service Technicians,0.882513,"[Robotics Technicians, Chemical Technicians, H...",[Company Description Strike provides end-to-en...,1,"[Test structures, controls, or mechanical, hyd...",1
32428,iOS Swift UI,tester,iOS Swift UI tester,Web Developers,0.636117,"[Desktop Publishers, Computer User Support Spe...","[Lead, you will be a technology professional a...",1,"[Design validation study features, such as sam...",1


In [48]:
final_df[final_df['similarity']>0.8]['job num'].sum()

np.int64(70092)

In [49]:
final_df[final_df['job num']>4]['job num'].sum()

np.int64(46778)

In [50]:
final_df[(final_df['job num']>1) & (final_df['similarity']>0.8)]['job num'].sum()

np.int64(61976)

In [52]:
filer_df = final_df[(final_df['job num']>1) & (final_df['similarity']>0.8)]
filer_df

,title_prefix,title_suffix,job title,occupation,similarity,occupation_candidate,job description,job num,task,task num
10067,Head of IT Infrastructure Technology,,Head of IT Infrastructure Technology,Architectural and Engineering Managers,0.800016,"[Regulatory Affairs Managers, Project Manageme...","[Manage IT Infrastructure Network & Systems, p...",3,"[Plan or direct the installation, testing, ope...",3
4337,Adobe Architect,,Adobe Architect,Database Architects,0.800018,"[Database Architects, Residential Advisors, Vi...",[Develop and execute advanced data collection ...,5,"[Collaborate with system architects, software ...",5
16113,Sr Engineering Manager - Data Marketplace role,,Sr Engineering Manager - Data Marketplace role,Data Warehousing Specialists,0.800030,"[General and Operations Managers, Project Mana...",[Managing Technology Vendors• Be an advocate t...,2,"[Perform system analysis, data analysis or pro...",2
10066,Specialized Trades Worker (OA FMDC 5/23),,Specialized Trades Worker (OA FMDC 5/23),First-Line Supervisors of Construction Trades ...,0.800065,[First-Line Supervisors of Construction Trades...,[Design and Construction has a job opportunity...,3,"[Create supporting documentation, such as meta...",3
16112,Lead Infusion Specialist - Registered Nurse fo...,,Lead Infusion Specialist - Registered Nurse fo...,Clinical Nurse Specialists,0.800080,"[Clinical Nurse Specialists, Health Informatic...",[Lead Franchise Nurse is disseminated to the s...,2,[Collaborate with other health care profession...,2
...,...,...,...,...,...,...,...,...,...,...
3260,Resource planning,manager,Resource planning manager,Facilities Managers,0.886209,"[Facilities Managers, Human Resources Speciali...",[Conduct research on trade invoice inquiries a...,5,"[Plan, administer, and control budgets for con...",5
15298,"PR Sr. Account Executive, Tech",sector,"PR Sr. Account Executive, Tech sector",Advertising and Promotions Managers,0.812939,"[Business Operations Specialists, All Other, A...",[Manage significant cycles of media inquiries ...,2,"[Test, validate, and reformulate models to ens...",2
5303,Tax Manager - C-Corp,specialist,Tax Manager - C-Corp specialist,Financial Risk Specialists,0.850468,"[Project Management Specialists, Financial Ris...",[Maintain and update the company's income tax ...,4,[Confer with customers and engineers to assess...,3
5082,Domain and DNS Management,specialist,Domain and DNS Management specialist,Computer Network Support Specialists,0.863204,"[Document Management Specialists, Data Warehou...","[Monitor and analyze network performance, iden...",4,[Troubleshoot network or connectivity problems...,4
